# Polygonal Area Light With Linearly Transformed Cosines

## Polygonal diffusion light with lambert diffuse model

Solved by Johann Heinrich Lambert in 18th century. 

$$
E(p_1, ..., p_n) = \frac 1 {2 \pi} \sum_{i=1}^{n} acos(\langle p_i, p_j \rangle) \langle {\frac {p_i \times p_j} {\| {p_i \times p_j} \|} }, {\begin{bmatrix} 0\\ 0\\ 1\\ \end{bmatrix}} \rangle \quad \textrm{where} \quad j = i + 1
$$


## Polygonal diffusion light for microfacet model

### Approximate GGX distribution and Smith shadowing term with linearly transformed cosines

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, colors
import ipywidgets as widgets


def func(a, b, c, d):
    grid_res_x = 64
    grid_res_y = grid_res_x

    # convention of scipy: theta is the azimuthal angle and phi is the polar angle
    phi = np.linspace(0, np.pi, num=grid_res_x)
    theta = np.linspace(0, 2*np.pi, num=grid_res_y)
    phi, theta = np.meshgrid(phi, theta)

    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    fig = plt.figure(figsize=[8, 8])

    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    trans = [[a, 0, b],
             [0, c, 0],
             [d, 0, 1]]
    inv_trans = np.linalg.inv(trans)

    # clamped cosine lobe
    dist = np.maximum(z, 0)

    x_copy = np.array(x)
    y_copy = np.array(y)
    z_copy = np.array(z)
    for row in range(grid_res_y):
        for colum in range(grid_res_x):
            px = x[row][colum]
            py = y[row][colum]
            pz = z[row][colum]

            new_pt = np.dot(inv_trans, [px, py, pz])
            new_pt = new_pt / np.linalg.norm(new_pt)

            x_copy[row][colum] = new_pt[0]
            y_copy[row][colum] = new_pt[1]
            z_copy[row][colum] = new_pt[2]

    dist = np.maximum(z_copy, 0)

    surf = ax.plot_surface(x, y, z, facecolors=cm.jet(dist), rcount=grid_res_x, ccount=grid_res_y)

    plt.show()

value_limit = 1
a = widgets.FloatSlider(min=0,max=value_limit,step=0.1,value=0.8)
b = widgets.FloatSlider(min=-1,max=value_limit,step=0.1,value=0.9)
c = widgets.FloatSlider(min=0,max=value_limit,step=0.1,value=0.2)
d = widgets.FloatSlider(min=0,max=value_limit,step=0.1,value=0)
widgets.interact(func, a=a, b=b, c=c, d=d)

A Jupyter Widget

<function __main__.func>

### [Approximate Fresnel term separately](http://blog.selfshadow.com/publications/s2016-advances/s2016_ltc_fresnel.pdf)

### Put together

HLSL code
```
    float theta = acos(dot(N, V));
    float2 uv = float2(roughness, theta / (0.5 * M_PIf));
    uv = uv * LUT_SCALE + LUT_BIAS;

    const float4 t = ltcMat.Sample(ltcSamp, coord);
    float3x3 invM = float3x3(
        float3(  1,   0, t.y),
        float3(  0, t.z,   0),
        float3(t.w,   0, t.x)
    );

    float res = ltcEvaluate(N, V, P, invM, points, false);

    /* Apply BRDF scale terms (BRDF magnitude and Schlick Fresnel) */
    const float2 schlick = ltcAmp.Sample(ltcSamp, uv).xy;
    res *= (F0 * schlick.x + (1.0 - F0) * schlick.y);
```

## References

* [Real-Time Polygonal-Light Shading with Linearly Transformed Cosines](https://eheitzresearch.wordpress.com/415-2/)
* [Real-Time Area Lighting: a Journey From Research to Production](http://blog.selfshadow.com/publications/s2016-advances/)
* [Real-Time Line- and Disk-Light Shading with Linearly Transformed Cosines](https://labs.unity.com/article/real-time-line-and-disk-light-shading-linearly-transformed-cosines)
* [paper](LTC/LTC.pdf)
* [slides](LTC/slides.pdf)
* [supplemental: MATLAB](LTC/supplemental_matlab.pdf)
* [WebGL Demo](http://blog.selfshadow.com/sandbox/ltc.html)
* [BRDF fitting code](LTC/fit.zip)